In [2]:
import pandas as pd
from scipy import stats
import random
from datetime import datetime as dt, timedelta
import numpy as np

import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

random.seed(42)

In [26]:
df_history = pd.read_csv('../data/sales_history.csv')
df_forecast = pd.read_csv('../data/sales_forecast.csv')
df_hub_stock = pd.read_csv('../data/hub_stock.csv')


In [29]:
# parameters: 
config = {
    "lead_time_hud2channel": 4,
    "lead_time_plant2hub": 40,
    "safety_stock_factor" : {
        "A" : 10,
        "B" : 10, 
        "C": 10
    },
    "channel_plan":{
        "channel 1" :{
            "priority" : 1,
            "service_level" : 0.8
        },        
        "channel 2" :{
            "priority" : 2,
            "service_level" : 0.8
        },
        "channel 3" :{
            "priority" : 3,
            "service_level" : 0.8
        },
        "channel 4" :{
            "priority" : 4,
            "service_level" : 0.8
        },
    }
}



sku_ls = ['A', 'B', 'C']
channel_ls = ['channel 1', 'channel 2', 'channel 3', 'channel 4']

current_date = '2023-01-14'
start_date = '2023-01-15'


import simulation
from importlib import reload 
reload(simulation)
import xlwings as xw

from simulation import generate_simulation,get_status,ttl_sales_through_rate,ttl_sales_shortage_rate

df_allocation = generate_simulation(
    config, 
    sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
)
# df_allocation.to_excel('../data/allocation.xlsx', index= False)

df_allocation['status'] = df_allocation.apply(get_status, axis = 1)
df_allocation['date'] = pd.to_datetime(df_allocation['date'])


df_status = pd.pivot_table(df_allocation, index=['sku_name', 'channel'],columns= 'date' ,values='status', aggfunc=lambda x: ', '.join(x))
sales_through_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: x.sales.sum()/(x['stock_in'].sum() + x[x['week']==0].stock) )
stockout_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: pd.Series(1 - x.sales.sum()/x.sales_potential_daily.sum(), index = ['stockout']))
sales_through_breakdown.columns = ['sales_through']
stockout_breakdown.columns = ['stockout']
df_status = sales_through_breakdown.join(stockout_breakdown).join(df_status)

# Connects to the active instance of Excel
wb = xw.Book('../data/allocation.xlsx')  # Connects to the active instance of Excel
sheet = wb.sheets['Sheet1']
sheet.range("A:L")[1:,:].clear_contents()
sheet['A1'].options(index=False).value = df_allocation
sheet = wb.sheets['Sheet2']
sheet.range('A3').value = df_status


sheet.range('A1').value = 'sale through rate:'
sheet.range('C1').value = 'stockout rate:'
sheet.range('A2').value = ttl_sales_through_rate(df_allocation)
sheet.range('C2').value = ttl_sales_shortage_rate(df_allocation)


/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/2679466548.py:57: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/2679466548.py:58: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [ ]:
# sales_demand * service_level     +    safety_stock 

# sales_demand * service_level     +    sales_demand/28 * factor

# sales_demand * (service_level + factor/28)

## Search

In [4]:
from itertools import permutations,product
from simulation import generate_simulation,get_status,ttl_sales_through_rate,ttl_sales_shortage_rate

In [30]:
search_space = {
    'lead_time_hud2channel' : [3,5],
    'lead_time_plant2hub' : [10,20],
    'priority' : [1,2,3,4], 
    'service_level' : [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
}


priority_permu = list(permutations(search_space['priority'], 4))


combinations = list(
    product(   
        search_space['lead_time_hud2channel'], 
        search_space['lead_time_plant2hub'], 
        search_space['service_level'], 
        priority_permu
    )
)

sku_ls = ['A', 'B', 'C']

config_spaces = [
    {
        "lead_time_hud2channel": c[0],
        "lead_time_plant2hub": c[1],
        "safety_stock_factor" : {
            "A" : 10,
            "B" : 10, 
            "C": 10
        },
        "channel_plan":{
            "channel 1" :{
                "priority" : c[3][0],
                "service_level" : c[2]
            },        
            "channel 2" :{
                "priority" : c[3][1],
                "service_level" : c[2]
            },
            "channel 3" :{
                "priority" : c[3][2],
                "service_level" : c[2]
            },
            "channel 4" :{
                "priority" : c[3][3],
                "service_level" : c[2]
            },
        }
    }
    for c in combinations
]

In [31]:
from tqdm import tqdm 

sales_through_ls = []
fill_rate_ls = []
for config_ in tqdm(config_spaces):

    df_allocation_ = generate_simulation(
        config_, 
        sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
    )
    
    sales_through_ls.append(ttl_sales_through_rate(df_allocation_))
    fill_rate_ls.append(1-  ttl_sales_shortage_rate(df_allocation_))


100%|██████████| 768/768 [00:35<00:00, 21.64it/s]


In [32]:
import plotly.express as px
fig = px.scatter(x=sales_through_ls, y=fill_rate_ls,hover_name = [i for i in range(len(config_spaces))],)

fig.update_xaxes(mirror = True, title = 'sale through')
fig.update_yaxes(mirror = True, title = 'fill rate')
fig.update_layout(
    template = 'simple_white', height = 600, width = 700,
)



In [33]:
config_spaces[398]

{'lead_time_hud2channel': 5,
 'lead_time_plant2hub': 10,
 'safety_stock_factor': {'A': 10, 'B': 10, 'C': 10},
 'channel_plan': {'channel 1': {'priority': 3, 'service_level': 0.6},
  'channel 2': {'priority': 2, 'service_level': 0.6},
  'channel 3': {'priority': 1, 'service_level': 0.6},
  'channel 4': {'priority': 4, 'service_level': 0.6}}}

In [35]:
import xlwings as xw

df_allocation = generate_simulation(
    config_spaces[398], 
    sku_ls,'2023-01-14', '2023-01-15', df_history, df_forecast, df_hub_stock, [1,2,3,4]
)
# df_allocation.to_excel('../data/allocation.xlsx', index= False)

df_allocation['status'] = df_allocation.apply(get_status, axis = 1)
df_allocation['date'] = pd.to_datetime(df_allocation['date'])


df_status = pd.pivot_table(df_allocation, index=['sku_name', 'channel'],columns= 'date' ,values='status', aggfunc=lambda x: ', '.join(x))
sales_through_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: x.sales.sum()/(x['stock_in'].sum() + x[x['week']==0].stock) )
stockout_breakdown = df_allocation.groupby(['sku_name','channel']).apply(lambda x: pd.Series(1 - x.sales.sum()/x.sales_potential_daily.sum(), index = ['stockout']))
sales_through_breakdown.columns = ['sales_through']
stockout_breakdown.columns = ['stockout']
df_status = sales_through_breakdown.join(stockout_breakdown).join(df_status)

# Connects to the active instance of Excel
wb = xw.Book('../data/allocation.xlsx')  # Connects to the active instance of Excel
sheet = wb.sheets['Sheet1']
sheet.range("A:L")[1:,:].clear_contents()
sheet['A1'].options(index=False).value = df_allocation
sheet = wb.sheets['Sheet2']
sheet.range('A3').value = df_status


sheet.range('A1').value = 'sale through rate:'
sheet.range('C1').value = 'stockout rate:'
sheet.range('A2').value = ttl_sales_through_rate(df_allocation)
sheet.range('C2').value = ttl_sales_shortage_rate(df_allocation)

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/148968180.py:14: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/var/folders/nx/d8ckq9qd7l3dz5plmv5gdj_40000gn/T/ipykernel_14531/148968180.py:15: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

